In [2]:
import sys
sys.path.append('..')

In [1]:
from pathos.multiprocessing import ProcessingPool as Pool
import numpy as np
import glob
from tqdm.notebook import tqdm
import os
import h5py
import json

In [ ]:
# mean = np.loadtxt('/mnt/disks/sdb/saved_data/ae_full_only_link/static_mean.csv')
# std = np.loadtxt('/mnt/disks/sdb/saved_data/ae_full_only_link/static_std.csv')
# np.set_printoptions(precision=3, suppress=True)
# print(np.stack([mean, std]).T)

In [ ]:
def save_mean_std_of_feats(static_feat_dir):
    print(static_feat_dir)
    obj_feats = np.load(static_feat_dir + '/static_feats.npz')

    stacked_feats = []
    for k in obj_feats.keys():
        feat = obj_feats[k]
        feat = feat.reshape([-1, feat.shape[-1]])
        stacked_feats.append(feat)
    stacked_feats = np.concatenate(stacked_feats)
    print(stacked_feats.shape, len(obj_feats.keys()))
    mean = stacked_feats.mean(0)
    std = stacked_feats.std(0)
    
    np.savetxt(static_feat_dir + '/static_mean.csv', mean, delimiter=",", fmt='%.6f')
    np.savetxt(static_feat_dir + '/static_std.csv', std, delimiter=",", fmt='%.6f')
    print('done')

save_mean_std_of_feats('/ccn2/u/rmvenkat/chpatel/test/ae_rn34_fnr/feats/')
# all_dirs = glob.glob('/mnt/disks/sdb/saved_data/*')
# for dr in all_dirs:
#     save_mean_std_of_feats(dr)

In [ ]:
def get_contact_label_from_h5(f):
    frames = list(f['frames'])
    target_contacted_zone = False
    for frame in reversed(frames):
        lbl = f['frames'][frame]['labels']['target_contacting_zone'][()]
        if lbl:  # as long as one frame touching, label is True
            target_contacted_zone = True
            break
    return target_contacted_zone

def get_contact_label_from_file(h5_fpath):
    with h5py.File(h5_fpath, 'r') as h5:
        lab = int(get_contact_label_from_h5(h5))
    return h5_fpath, lab
        
def save_readout_labels(scenario):
    out_path = f'/mnt/fs5/chpatel/saved_data/dataset/ocp_labels_{scenario}.json'
    data_root = '/mnt/fs3/rmvenkat/data/physion_train_data'
    subsets = ['dynamics_training', 'readout_training', 'model_testing']

    out_path = f'/ccn2/u/rmvenkat/chpatel/saved_data/dataset_v2/ocp_labels_{scenario}.json'
    data_root = '/ccn2/u/chpatel/physion_v2_hlink/'
    subsets = ['dynamics_training']

    ret = {}
    for ss in tqdm(subsets):
        all_hdf5 = glob.glob(os.path.join(data_root, ss, scenario, '*.hdf5'))
        
        out = Pool(8).map(get_contact_label_from_file, all_hdf5)
        for k, v in out:
            ky = ss + '/' + os.path.basename(k)
            ret[ky] = v

    with open(out_path, 'w') as f:
        json.dump(ret,  f)
    return ret


SCENARIOS = [
#     'Collide',
    'Dominoes',
    'Drop',
#     'Roll',
#     'Contain',
#     'Drape',
#     'Link',
#     'Support',
]

for scen in SCENARIOS:
    ret = save_readout_labels(scen)

In [ ]:
def gen_hlinks(src_dir, dest_dir):
    os.makedirs(dest_dir, exist_ok=False)
    all_hdf5 = glob.glob(os.path.join(src_dir, '*/*.hdf5'))
    for hf in all_hdf5:
        tokens = hf.split('/')
        lnd = os.path.join(dest_dir, tokens[-2] + '_' + tokens[-1])
        print(hf, lnd)
        os.symlink(hf, lnd)

src_dir = "/ccn2/u/rmvenkat/data/testing_physion/all_sce_all_fixed/lf_0/dominoes_all_movies/train"
dest_dir = "/ccn2/u/chpatel/physion_v2_hlink/dynamics_training/Dominoes"
# gen_hlinks(src_dir, dest_dir)